In [1]:
import os
import gym
import gym_donkeycar
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2 as cv
import time
import pickle
import birds_eye_vector_space
import random
from pandas import Series, DataFrame
from collections import deque

#from keras.layers import Dense
#from tensorflow.keras.optimizers import Adam
#from keras.initializers import normal, identity
#from keras.models import model_from_json
#from keras.models import Sequential
#from keras.layers.core import Dense, Dropout, Activation, Flatten
#from keras.layers.convolutional import Convolution2D, MaxPooling2D

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.client import device_lib
#from keras import backend as K
#import tensorflow.keras.backend as K

C:\Users\studwilksa2535\AppData\Roaming\Python\Python38\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:18: DeprecationWarning: Call to deprecated create function FileDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  DESCRIPTOR = _descriptor.FileDescriptor(
C:\Users\studwilksa2535\AppData\Roaming\Python\Python38\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:36: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _descriptor.FieldDescriptor(
C:\Users\studwilksa2535\AppData\Roaming\Python\Python38\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:29: DeprecationWarning: Call to deprecated create function Descriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find d

In [2]:
# use gpu
#K.tensorflow_backend._get_available_gpus()
#sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
#K.tensorflow_backend._get_available_gpus()

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  2


In [4]:
tf.test.is_built_with_cuda()

True

In [5]:
tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [6]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

In [7]:
# run on the second GPU
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
#os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [8]:
len(tf.config.list_physical_devices('GPU'))
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 3919986167459309486
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9401860096
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 5262928906516595001
 physical_device_desc: "device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:49:00.0, compute capability: 7.5"
 xla_global_id: 416903419,
 name: "/device:GPU:1"
 device_type: "GPU"
 memory_limit: 6271991808
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 8136955003031579519
 physical_device_desc: "device: 1, name: GeForce RTX 2070 SUPER, pci bus id: 0000:21:00.0, compute capability: 7.5"
 xla_global_id: 2144165316]

In [9]:
#mirrored_strategy = tf.distribute.MirroredStrategy()

In [10]:
#sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [11]:
# Supress scientific notation like: e*+03
np.set_printoptions(suppress=True)

In [12]:
ANCHOR_POINTS = 8
ROAD_ROI = np.array([(120,90),(200,90),(0,200),(320,200)],dtype='float32')
WARPED_IMAGE_SHAPE = np.array([[10,320],[0,0],[200,0],[200,310]],np.int32)            # NEW IMAGE Shape after Warping !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
WARPED_IMAGE_HEIGHT = 320
WARPED_IMAGE_WIDTH = 200
NUMBER_OF_DEPTH_LAYERS = 50

VECTOR_SPACE_IMAGE_ROWS = 50
VECTOR_SPACE_IMAGE_COLUMNS = 100
VECTOR_SPACE_IMAGE_CHANNELS = 4 # 4*(50,100) stacked frames 

In [13]:
# function to save the plots
def save_plot(data, name, episode):
    plt.figure(figsize=(8,5), frameon=True)
    #plt.plot([episode for episode in range(len(data))], data)
    plt.plot([ep for ep in range(episode)], data)
    plt.xlabel('Episodes')
    plt.ylabel(name)
    figplot = '%s-episode-%s.png' % (name,episode)
    #figplot = '{}-episode-{}.png'.format(name,episode)
    #location = '{}/saves/torch/{}'.format(os.getcwd(),figplot)
    #location = '%s/collected_data/plots/%s' % (os.getcwd(),figplot)
    location = 'C:\\Users\\studwilksa2535\\Desktop\\DonkeyCarAI\\collected_data\\plots\\%s' % figplot 
    plt.savefig(location, transparent=False)
    plt.close()

In [14]:
def save_data_as_dataframe(episode_number, reward, loss, measured_time, max_cte, average_cte, last_lap_time):
    episodes = [i for i in range(1, episode_number+1)]
    collected_data = {'episode': episodes, 'reward': reward, 'loss':loss, 'time': measured_time, 'max_cte': max_cte, 'average_cte': average_cte, 'lap_time': last_lap_time}
    df_data = DataFrame.from_dict(collected_data).set_index('episode')
    
    df_name = 'data-episode-%s.pkl' % episode
    #location = '%s/collected_data/raw_data/%s' % (os.getcwd(),df_name)
    location = 'C:\\Users\\studwilksa2535\\Desktop\\DonkeyCarAI\\collected_data\\raw_data\\%s' % df_name
    df_data.to_pickle(location)

In [15]:
vector_space = birds_eye_vector_space.Vector_space(ANCHOR_POINTS, ROAD_ROI, WARPED_IMAGE_SHAPE, WARPED_IMAGE_HEIGHT, WARPED_IMAGE_WIDTH, NUMBER_OF_DEPTH_LAYERS)

vector space is initialized
loading camera parameters


In [16]:
os.getcwd()

'C:\\Users\\studwilksa2535\\Desktop\\DonkeyCarAI\\Master-Thesis-Development-of-a-Deep-RL-Model-for-simulated-Driving-2D-Vector-Space\\docker\\src'

In [17]:
class DQNAgent:
    
    def __init__(self, state_size, action_size):
        self.t = 0
        self.max_Q = 0
        self.train = True
        # Set to True to train on images with segmented lane lines
        self.lane_detection = False
        
        # Huber loss
        self.huber_loss = tf.keras.losses.Huber(reduction=tf.keras.losses.Reduction.SUM)
        
        # Get size of state and action
        self.state_size = state_size
        self.action_size = action_size
        
        # CNN Input
        #self.input_shape = (VECTOR_SPACE_IMAGE_CHANNELS, VECTOR_SPACE_IMAGE_ROWS, VECTOR_SPACE_IMAGE_COLUMNS, 4)   # == 4 * (50, 100, 1); keras: (1,50,100,4)
        self.input_shape = (1, VECTOR_SPACE_IMAGE_ROWS, VECTOR_SPACE_IMAGE_COLUMNS, VECTOR_SPACE_IMAGE_CHANNELS)   # == 4 * (50, 100, 1); keras: (1,50,100,4)
        
        # These are hyper parameters for the DQN
        self.discount_factor = 0.99
        self.learning_rate = 1e-4
        if (self.train):
            self.epsilon = 1.0
            self.initial_epsilon = 1.0
        else:
            self.epsilon = 1e-6
            self.initial_epsilon = 1e-6
        self.epsilon_min = 0.02
        self.batch_size = 512
        self.train_start = 100
        self.explore = 10000
        
        # Create replay memory using deque
        self.memory = deque(maxlen=10000)
        
        # Create main model and target model                # Double DQN !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        self.model = self.build_model()
        self.target_model = self.build_model()

        # Copy the model to target model
        # --> initialize the target model so that the parameters of model & target model to be same
        self.update_target_model()
        
    def build_model_old(self):
        print('delte this')
        #model = Sequential()
        #model.add(Convolution2D(32, 8, 8, subsample=(4, 4), border_mode='same',input_shape=(VECTOR_SPACE_IMAGE_ROWS,VECTOR_SPACE_IMAGE_COLUMNS,VECTOR_SPACE_IMAGE_CHANNELS)))  #80*80*4
        #model.add(Activation('relu'))
        #model.add(Convolution2D(64, 4, 4, subsample=(2, 2), border_mode='same'))
        #model.add(Activation('relu'))
        #model.add(Convolution2D(64, 3, 3, subsample=(1, 1), border_mode='same'))
        #model.add(Activation('relu'))
        #model.add(Flatten())
        #model.add(Dense(512))
        #model.add(Activation('relu'))

        ## 15 categorical bins for Steering angles
        #model.add(Dense(15, activation="linear")) 

        #adam = Adam(lr=self.learning_rate)
        #model.compile(loss='mse',optimizer=adam)

        #return model

    def build_model(self):
        model = keras.Sequential([
            layers.Conv2D(filters=32, strides=(4, 4),kernel_size=8, padding='same', activation='relu', input_shape=(self.input_shape[1:])),
            layers.Conv2D(filters=64, strides=(2, 2),kernel_size=4, padding='same', activation='relu'),
            layers.Conv2D(filters=64, strides=(1, 1),kernel_size=3, padding='same', activation='relu'),
            layers.Flatten(),
            layers.Dense(units=512,activation='relu'),
            layers.Dense(units=512,activation='relu'), # TODO ADDED THIS LAYER FOR TESTING
            layers.Dense(units=15,activation='linear'),
        ])
        #optimizer = tf.keras.optimizers.Adam(learning_rate=self.learning_rate)
        optimizer = tf.keras.optimizers.Adam()
        #optimizer = tf.keras.optimizers.SGD()
        model.compile(loss='mse', optimizer=optimizer)
        #model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=optimizer)
        return model
    
    def calculate_loss(self, y_true, y_pred):    
        y_pred = tf.convert_to_tensor_v2(y_pred)
        y_true = tf.cast(y_true, y_pred.dtype)
        return tf.reduce_mean(math_ops.square(y_pred - y_true), axis=-1)
    
    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    # Get action from model using epsilon-greedy policy
    def get_action(self, s_t):
        if np.random.rand() <= self.epsilon:
            #print("Return Random Value")
            #return random.randrange(self.action_size)
            return np.random.uniform(-1,1)
        else:
            #print("Return Max Q Prediction")
            q_value = self.model.predict(s_t)
            # Convert q array to steering value
            return linear_unbin(q_value[0])

    def replay_memory(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        #if self.epsilon > self.epsilon_min:
        #    #self.epsilon *= self.epsilon_decay
        #    self.epsilon -= (self.initial_epsilon - self.epsilon_min) / self.explore

    def train_replay(self):
        if len(self.memory) < self.train_start:
            return
        
        batch_size = min(self.batch_size, len(self.memory))
        minibatch = random.sample(self.memory, batch_size)

        state_t, action_t, reward_t, state_t1, terminal = zip(*minibatch)
        #print('Starting training\nstate_t: %s\naction_t: %s\nreward_t: %s\nstate_1: %s\nterminal: %s' % (len(state_t), len(action_t), len(reward_t), len(state_t1), len(terminal)))
        state_t = np.concatenate(state_t)
        state_t1 = np.concatenate(state_t1)
        #print('Concat:\nstate: %s\nstate_t1: %s' % (state_t, state_t1))
        targets = self.model.predict(state_t)
        self.max_Q = np.max(targets[0])
        target_val = self.model.predict(state_t1)
        target_val_ = self.target_model.predict(state_t1)
        for i in range(batch_size):
            if terminal[i]:
                targets[i][action_t[i]] = reward_t[i]
            else:
                a = np.argmax(target_val[i])
                targets[i][action_t[i]] = reward_t[i] + self.discount_factor * (target_val_[i][a])

        
        # train on the current batch and return a dictionary with the loss and so on
        #metrics = self.model.train_on_batch(x=state_t, y=targets, reset_metrics=True, return_dict=True)           
        metrics = self.model.train_on_batch(x=state_t, y=targets, reset_metrics=True, return_dict=True) #reset Metrix different
        return metrics['loss']
        
    def load_model(self, name):
        self.model.load_weights(name)

    # Save the model which is under training
    def save_model(self, name):
        self.model.save_weights(name)

In [18]:
def linear_bin(a):
    """
    Convert a value to a categorical array.

    Parameters
    ----------
    a : int or float
        A value between -1 and 1

    Returns
    -------
    list of int
        A list of length 15 with one item set to 1, which represents the linear value, and all other items set to 0.
    """
    a = a + 1
    b = round(a / (2 / 14))
    arr = np.zeros(15)
    arr[int(b)] = 1
    return arr

In [19]:
def linear_unbin(arr):
    """
    Convert a categorical array to value.

    See Also
    --------
    linear_bin
    """
    if not len(arr) == 15:
        raise ValueError('Illegal array length, must be 15')
    b = np.argmax(arr)
    a = b * (2 / 14) - 1
    return a

## Main training loop

In [20]:
#%% SET UP ENVIRONMENT
# Normal one 
#os.environ['DONKEY_SIM_PATH'] = "/home/zamy/masterthesis/DonkeySimLinux/donkey_sim.x86_64"
os.environ['DONKEY_SIM_PATH'] = "C:\\Users\\studwilksa2535\\Desktop\\DonkeyCarAI\\test\\DonkeySimWin\\donkey_sim.exe"
os.environ['DONKEY_SIM_PORT'] = str(9091)
os.environ['DONKEY_SIM_HEADLESS'] = str(0) # "1" is headless

CAMERA_CONF = {'cam_config':{'img_w': '320',
                        'img_h': '200',
                        'img_d': '1',   # 3 for colored Tensor image
                        'img_enc': 'PNG', 
                        'fov': '90', 
                        'fish_eye_x': '0.0', 
                        'fish_eye_y': '0.0', 
                        'offset_x': '0.0', 
                        'offset_y': '0.0', 
                        'offset_z': '0.0', 
                        'rot_x': '0'}}
# Other tracks
#env = gym.make("donkey-generated-track-v0",conf=CAMERA_CONF)

# TRACKS TRACKS - TKarmer Tracks
#env = gym.make("donkey-generated-track-v0",conf=CAMERA_CONF) #,conf=config
# Random track, but the reward is kinda not working, IF THE REWARD WORKS IT IS KINDA RANDOM !!, this maybe debends on the environment
env = gym.make("donkey-generated-roads-v0",conf=CAMERA_CONF) #,conf=config

# Mini Monaco Track
#env = gym.make("donkey-minimonaco-track-v0",conf=CAMERA_CONF) #,conf=config
#env = gym.make("donkey-generated-track-v0") #,conf=config
#env = gym.wrappers.ResizeObservation(env,(200,320))

## DELETE LATER
#env.frameskip = 1
#gym.wrappers.max

# Create DQN Model
# Get size of state and action from environment
state_size = (VECTOR_SPACE_IMAGE_ROWS, VECTOR_SPACE_IMAGE_COLUMNS, VECTOR_SPACE_IMAGE_CHANNELS)
action_size = 15 # Steering and Throttle
throttle = 0.15 # Set the throttle as a constant value
agent = DQNAgent(state_size, action_size)


# Arrays for data collection / exploratory data analysis
average_cte = []
average_speed = []
measured_time = []
average_rewards = []
average_loss = []
collected_max_cte = []
collected_lap_time = []

# when a model and the data should be saved
save_state = 200

for episode in range(1,7001):
    # Resetting the environment and preprocessing the first image
    observation = env.reset()
    # Graykonvertion of the observation image
    observation = cv.cvtColor(observation, cv.COLOR_BGR2GRAY)
    observation = vector_space.image_preprocessing(observation)
    # frame stacking (4 times)
    obv_stack = np.stack((observation, observation, observation, observation), axis= 2)
    # reshaping for keras
    obv_stack = obv_stack.reshape(1, obv_stack.shape[0], obv_stack.shape[1], obv_stack.shape[2])
    
    #action = np.array([0,0.10]) # drive straight with small speed
    #action = np.array([1,0.1]) # drive straight with small speed
    
    # summed up values for data collection
    # cumulative values for data collection
    total_cte = float(0)
    total_speed = float(0)
    total_reward = float(0)
    total_time = time.time()
    total_loss = float(0)
    max_cte = int(0)
    lap_time = int(0)
        
    # counting the amount of frames per episode
    frames = int(0)
    
    # boolean that describes if the env is done with this episode
    done = False
    while not done:
        # incrementing the amount of frames per episode
        frames += 1
        
        # Making a prediction for the current state
        # and getting the information form the next step
        steering = agent.get_action(obv_stack)
        action = [steering, throttle]
        next_observation, reward, done, info = env.step(action)
        
        current_cte = abs(info['cte'])
        # punish and reward for a low or high 'CTE' with the logarithmic function
        if current_cte < .25:
            reward += abs(np.log(.10)) * 8  # 4 before
        elif current_cte >= .25 and current_cte < .5:
            reward += current_cte * 2
        elif current_cte >= .5 and current_cte < 1: 
            reward += abs(np.log(current_cte))
        elif current_cte >= 1 and current_cte < 2: 
            reward -= abs(current_cte) * 2
        elif current_cte >= 2 and current_cte < 3: 
            reward -= abs(current_cte) * 3
        elif current_cte >= 3 and current_cte < 4: 
            reward -= abs(current_cte) * 4
        elif current_cte >=4:
            reward -= abs(current_cte) * 5
                    
        # if the car leaves the track punish it
        if done:
            reward -= 100
        # Check if the CTE of the environment is bugged
        if episode == 1:
            #print('CTE: %s\tReward: %s' % (info['cte'], reward)
            print('REWARD: %s\tCTE: %s' % (reward, current_cte))
        
        # Graykonvertion of the observation image
        next_observation = cv.cvtColor(next_observation, cv.COLOR_BGR2GRAY)
        
        # preprocessing the new observation
        next_observation_show = vector_space.image_preprocessing(next_observation)
        # reshaping for keras
        #next_observation = next_observation_show.reshape(1, next_observation_show.shape[0], next_observation_show.shape[1], 1)
        # appending to the observation stack
        #obv_stack = np.append(next_observation, obv_stack[:, :, :, :3], axis=3)
        
        obv_stack_t1 = np.stack([next_observation_show, obv_stack[0,:,:,0],obv_stack[0,:,:,1],obv_stack[0,:,:,2]], axis=2)
        obv_stack_t1 = obv_stack_t1.reshape(1, obv_stack_t1.shape[0], obv_stack_t1.shape[1], obv_stack_t1.shape[2])
                
        # saving the sample <s, a, r, s'> to the replay memory
        agent.replay_memory(obv_stack, np.argmax(linear_bin(steering)), reward, obv_stack_t1, done)        
        
        # training the DDQN, if training is enabled
        #if agent.train:
        #    loss = agent.train_replay()
        #    if loss != None:
        #        total_loss += loss
        #        #print('loss: %s ' % (total_loss / frames))
        
        # overwriting the stack and incrementing the time/frame counter
        obv_stack = obv_stack_t1
        agent.t += 1       
        
        # adding up the collected data
        current_cte = info['cte'] 
        total_cte += current_cte
        total_speed += info['speed']
        total_reward += reward
        # update if there is any change in incrementation 
        if max_cte < current_cte:
            max_cte = current_cte
        if lap_time < info['last_lap_time']:
            lap_time = info['last_lap_time']
            
        cv.imshow('vec img', next_observation_show)
        
        # stop the training if the car passes 120 seconds,
        # so there is no infinit learning
        if abs(total_time - time.time()) >= 120:
            done = True

        # Graykonvertion of the observation image
        #observation = cv.cvtColor(observation, cv.COLOR_BGR2GRAY)
        
        # Image preprocessing
        #vec_image = vector_space.image_preprocessing(observation)
        
        #print('REWARD: %s' % reward)
        
        #cv.imshow('vec img', vec_image)
        
        #break       # TODO: DELTE LATER
        if cv.waitKey(25) & 0xFF == ord('q'):
            cv.destroyAllWindows()
            break
    
    # Training after each episode
    # do 10 training sets of each with a batch size of 512
    for i in range(0,10):
        if agent.train:
            total_loss += agent.train_replay()
            #total_loss = agent.train_replay()
            #if loss != None:
            #    total_loss = loss
    
    
    # AFTER this Episode & the environment returns True for the done variable
    # updating the target DDQN
    agent.update_target_model()
    
    # calculating the average loss in this episode
    #episode_loss = total_loss.copy() #/ frames
    total_loss = total_loss / 512 * 10
    episode_cte = total_cte / frames
    episode_reward = total_reward / frames
    
    # updating the time parameter
    total_time = abs(total_time - time.time())
    
    # appending the collected data
    average_cte.append(episode_cte)
    average_speed.append(total_speed / frames)
    measured_time.append(total_time)
    average_rewards.append(episode_reward)
    average_loss.append(total_loss)
    collected_max_cte.append(max_cte)
    collected_lap_time.append(lap_time) 
    
    # Testing of saving the plot
    #if episode % 3 == 0 and episode != 0:
    #    save_plot(colleted_rewards, 'Reward', episode)
    #    cv.destroyAllWindows()
    #    break
    
    # Print episode information    
    print('EPISODE: %s | TIME: %s s | REWARD: %s | FRAMES: %s | QMAX: %s | EPSILON: %s | CTE: %s | LOSS: %s' % (episode, total_time, episode_reward, frames, str(agent.max_Q), agent.epsilon, episode_cte, total_loss))    
    
    # update the epsilon after each episode
    # Epsilion decay over time/amount of trainings
    if agent.epsilon >= agent.epsilon_min:
        #self.epsilon *= self.epsilon_decay
        #self.epsilon -= (self.initial_epsilon - self.epsilon_min) / self.explore
        #self.epsilon -= 0.00025
        agent.epsilon -= 0.00025 # before 0.00035
    
    if episode % save_state == 0 and episode != 0:
        print('saving after %s episodes' % save_state)
        # saving the model
        agent.save_model('C:\\Users\\studwilksa2535\\Desktop\\DonkeyCarAI\\models\\model_episode_%s.h5' % episode)        
        
        # Saving the data as plots
        save_plot(average_rewards, 'Reward', episode)
        save_plot(average_cte, 'Average cte', episode)
        save_plot(collected_max_cte, 'Max cte', episode)
        save_plot(measured_time, 'Time', episode)
        save_plot(average_loss, 'Loss', episode)
        save_plot(collected_lap_time, 'Lap Time', episode)
        
        # Saving the collected data
        save_data_as_dataframe(episode, average_rewards, average_loss, measured_time, collected_max_cte, average_cte, collected_lap_time)
    
    # closing all cv windows
    cv.destroyAllWindows()

# closing all cv windows
cv.destroyAllWindows()
# Close the enviroment
env.close()

INFO:gym_donkeycar.core.client:connecting to localhost:9091 
C:\Users\studwilksa2535\AppData\Roaming\Python\Python38\site-packages\gym\spaces\box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


starting DonkeyGym env
Setting default: start_delay 5.0
Setting default: max_cte 5.0
Setting default: frame_skip 1
Setting default: cam_resolution (120, 160, 3)
Setting default: log_level 20
Setting default: host localhost
Setting default: port 9091
loading scene generated_road


INFO:gym_donkeycar.envs.donkey_sim:on need car config
INFO:gym_donkeycar.envs.donkey_sim:sending car config.
INFO:gym_donkeycar.envs.donkey_sim:done sending cam config. {'img_w': '320', 'img_h': '200', 'img_d': '1', 'img_enc': 'PNG', 'fov': '90', 'fish_eye_x': '0.0', 'fish_eye_y': '0.0', 'offset_x': '0.0', 'offset_y': '0.0', 'offset_z': '0.0', 'rot_x': '0'}


REWARD: 18.420681812369217	CTE: 0.0001457528
REWARD: 18.481999771440407	CTE: 0.0002334005
REWARD: 18.561648014777607	CTE: 0.0004656383
REWARD: 18.635571457706316	CTE: 5.55217e-05
REWARD: 18.692860878982195	CTE: 0.0003189774
REWARD: 18.764945355910807	CTE: 0.001091771
REWARD: 18.833987822923945	CTE: 0.003299495
REWARD: 18.905784418150922	CTE: 0.00361955
REWARD: 18.97778016427459	CTE: 0.004058632
REWARD: 19.040146317013978	CTE: 0.002120501
REWARD: 19.0877820866723	CTE: 0.002571429
REWARD: 19.158030892631523	CTE: 0.003224008
REWARD: 19.22453497443656	CTE: 0.005461248
REWARD: 19.286871241772662	CTE: 0.006219534
REWARD: 19.355202351226314	CTE: 0.006262695
REWARD: 19.42222045107615	CTE: 0.005750977
REWARD: 19.45697416535147	CTE: 0.01089771
REWARD: 19.52367247880229	CTE: 0.01633491
REWARD: 19.583585166958876	CTE: 0.02293821
REWARD: 19.6419200377083	CTE: 0.03186431
REWARD: 19.703060975004142	CTE: 0.03987719
REWARD: 19.75113642128334	CTE: 0.04667311
REWARD: 19.804080166865532	CTE: 0.06332527
RE

REWARD: -0.7450311833679999	CTE: 1.750888
REWARD: -0.7455518753224006	CTE: 1.758834
REWARD: -0.7495969437232	CTE: 1.767859
REWARD: -0.7855727897600002	CTE: 1.785792
REWARD: -0.8298110219999999	CTE: 1.8043
REWARD: -0.8512130852607998	CTE: 1.814908
REWARD: -0.843268039661	CTE: 1.818595
REWARD: -0.9325585628719999	CTE: 1.83402
REWARD: -0.9902295107192001	CTE: 1.861107
REWARD: -1.0756525704179998	CTE: 1.880115
REWARD: -1.0796576373760005	CTE: 1.877312
REWARD: -1.0216259841706	CTE: 1.860911
REWARD: -1.1053010300204003	CTE: 1.869318
REWARD: -1.0723277155370003	CTE: 1.866297
REWARD: -1.0638579538960005	CTE: 1.859213
REWARD: -0.991444569724	CTE: 1.83977
REWARD: -0.9007515976282003	CTE: 1.815059
REWARD: -0.8547855547102001	CTE: 1.798271
REWARD: -0.8220233596310003	CTE: 1.792245
REWARD: -0.7998697785359998	CTE: 1.790703
REWARD: -0.7903882750846001	CTE: 1.794257
REWARD: -0.7751043526744006	CTE: 1.794078
REWARD: -0.8479545296095998	CTE: 1.819896
REWARD: -0.8743387050520002	CTE: 1.829549
REWARD: -0

...

EPISODE: 4925 | TIME: 23.75857138633728 s | REWARD: 12.834767363208888 | FRAMES: 95 | QMAX: 973.25006 | EPSILON: 0.019750000000082396 | CTE: 0.47853991081894737 | LOSS: 23.104638010263443
EPISODE: 4926 | TIME: 37.11790585517883 s | REWARD: 0.8206970673258515 | FRAMES: 178 | QMAX: 964.3003 | EPSILON: 0.019750000000082396 | CTE: -1.7336706205112362 | LOSS: 20.664930045604706
EPISODE: 4927 | TIME: 105.4043390750885 s | REWARD: -0.0524987998939487 | FRAMES: 623 | QMAX: 867.7815 | EPSILON: 0.019750000000082396 | CTE: -2.119033328998395 | LOSS: 24.477473497390747
EPISODE: 4928 | TIME: 50.762115478515625 s | REWARD: -5.58319176310856 | FRAMES: 268 | QMAX: 1016.4191 | EPSILON: 0.019750000000082396 | CTE: -2.6340900790358948 | LOSS: 30.49420103430748
EPISODE: 4929 | TIME: 61.61822319030762 s | REWARD: 14.858331770663803 | FRAMES: 336 | QMAX: 1010.8015 | EPSILON: 0.019750000000082396 | CTE: -0.009097657016071433 | LOSS: 24.89893466234207
EPISODE: 4930 | TIME: 117.778329372406 s | REWARD: 6.53154

EPISODE: 4969 | TIME: 35.15970516204834 s | REWARD: 6.978648273679361 | FRAMES: 168 | QMAX: 862.45715 | EPSILON: 0.019750000000082396 | CTE: -1.3480799585645413 | LOSS: 29.936323761940002
EPISODE: 4970 | TIME: 35.425084829330444 s | REWARD: -5.444718844721711 | FRAMES: 168 | QMAX: 974.62537 | EPSILON: 0.019750000000082396 | CTE: -2.5481929598339286 | LOSS: 25.787047892808914
EPISODE: 4971 | TIME: 129.2155797481537 s | REWARD: 6.082801615415832 | FRAMES: 774 | QMAX: 817.5 | EPSILON: 0.019750000000082396 | CTE: -1.1135717076459948 | LOSS: 27.125468999147415
EPISODE: 4972 | TIME: 129.19400906562805 s | REWARD: 7.629751169831789 | FRAMES: 772 | QMAX: 1042.0768 | EPSILON: 0.019750000000082396 | CTE: -1.089914211109845 | LOSS: 26.230122298002243
EPISODE: 4973 | TIME: 47.54575204849243 s | REWARD: 15.554806570372545 | FRAMES: 246 | QMAX: 1038.9321 | EPSILON: 0.019750000000082396 | CTE: 0.00965925974796752 | LOSS: 26.950426995754242
EPISODE: 4974 | TIME: 56.115387201309204 s | REWARD: 10.03393

EPISODE: 5013 | TIME: 129.20817875862122 s | REWARD: 14.710014658138935 | FRAMES: 769 | QMAX: 1094.46 | EPSILON: 0.019750000000082396 | CTE: -0.12006429992561764 | LOSS: 22.792778462171555
EPISODE: 5014 | TIME: 129.47857284545898 s | REWARD: 13.06353098677988 | FRAMES: 771 | QMAX: 961.4289 | EPSILON: 0.019750000000082396 | CTE: -0.14341618780804155 | LOSS: 25.037303268909454
EPISODE: 5015 | TIME: 17.49924945831299 s | REWARD: 8.598988650573299 | FRAMES: 47 | QMAX: 880.662 | EPSILON: 0.019750000000082396 | CTE: 0.8135195053404254 | LOSS: 20.349824130535126
EPISODE: 5016 | TIME: 129.33992671966553 s | REWARD: 5.198046289971885 | FRAMES: 759 | QMAX: 816.61316 | EPSILON: 0.019750000000082396 | CTE: -0.9422498446574438 | LOSS: 24.81858253479004
EPISODE: 5017 | TIME: 54.92471623420715 s | REWARD: -2.013884316468388 | FRAMES: 289 | QMAX: 933.1837 | EPSILON: 0.019750000000082396 | CTE: -2.042942165555711 | LOSS: 21.663250029087067
EPISODE: 5018 | TIME: 38.614240884780884 s | REWARD: 1.82585329

EPISODE: 5057 | TIME: 24.862768411636353 s | REWARD: 10.18770441424675 | FRAMES: 99 | QMAX: 951.85394 | EPSILON: 0.019750000000082396 | CTE: 0.48106023402048487 | LOSS: 29.51739564538002
EPISODE: 5058 | TIME: 66.10987758636475 s | REWARD: 9.590980095157525 | FRAMES: 361 | QMAX: 919.87286 | EPSILON: 0.019750000000082396 | CTE: -0.6868422760357618 | LOSS: 24.941098988056183
EPISODE: 5059 | TIME: 80.88647127151489 s | REWARD: 10.046320966346347 | FRAMES: 452 | QMAX: 911.2211 | EPSILON: 0.019750000000082396 | CTE: 0.579890583960177 | LOSS: 33.00824448466301
EPISODE: 5060 | TIME: 57.61841559410095 s | REWARD: 8.578694143854722 | FRAMES: 307 | QMAX: 1043.9315 | EPSILON: 0.019750000000082396 | CTE: 0.6649802104784035 | LOSS: 27.915456891059875
EPISODE: 5061 | TIME: 57.88047909736633 s | REWARD: 12.31693760835923 | FRAMES: 314 | QMAX: 995.1466 | EPSILON: 0.019750000000082396 | CTE: 0.19837439320605096 | LOSS: 26.92952334880829
EPISODE: 5062 | TIME: 129.39936685562134 s | REWARD: 9.872396326847

KeyboardInterrupt: 